In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image

from tensorflow.keras import Sequential, layers, models
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16
from tensorflow.keras.preprocessing.image import ImageDataGenerator


import matplotlib.pyplot as plt

import os
from os import walk

In [ ]:
# Load DF
df_path = "path/to/df.csv"
df = pd.read_csv(df_path)

In [ ]:
data_path = 'path/to/data'
f = []
for (dirpath, dirnames, filenames) in walk(data_path):
    f.extend(filenames)
    break

In [ ]:
images_ids = []
for filename in filenames:
    images_ids.append(int(filename.strip('.png')))

In [ ]:
y = []
for image_id in images_ids:
    #print(id)
    #y.append(int(df_masked_cat[df_masked_cat['Id'] == image_id].Demand.values[0]))
    #print(int(df_masked_cat[df_masked_cat['Id'] == image_id].Demand.values[0]))
    y.append(int(df_masked_cat[df_masked_cat['Id'] == image_id].Demand.values[0]))

In [ ]:
def load_satellite_images(data_path, data_folder, y):
        
    data_path = data_path
    y = np.array(y)
    imgs = []
    images_path = [os.path.join(data_folder, elt) for elt in os.listdir(os.path.join(data_path)) if elt.find('.png')>0]
    
    for path in images_path:
        if os.path.exists(path):
            image = Image.open(path)
            #image = image.resize((256, 256))
            imgs.append(np.array(image))

    X = np.array(imgs)
    X_train, X_sub, y_train, y_sub = train_test_split(X, y, test_size=0.40, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_sub, y_sub, test_size=0.20, random_state=42)

    return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
data_path = 'data/high_res'
data_folder = 'data/high_res'
X_train, y_train, X_val, y_val, X_test, y_test = load_satellite_images(data_path, data_folder, y)

In [ ]:
#Preprocess to match VGG16 or resnet50

model_selected = 'vgg16'

if model_selected == 'vgg16':
  X_train = preprocess_input_vgg16(X_train) 
  X_val = preprocess_input_vgg16(X_val)
  X_test = preprocess_input_vgg16(X_test)

elif model_selected == 'resnet50':
  X_train = preproc_res(X_train) 
  X_val = preproc_res(X_val)
  X_test = preproc_res(X_test)


In [ ]:
#load model
def load_model(model_selected):

    model = Sequential()

    if model_selected == 'vgg16':
      model = VGG16(weights="imagenet", include_top=False, input_shape=X_train[0].shape)
    elif model_selected == 'resnet50':
      model = ResNet50(weights='imagenet', include_top = False, input_shape=X_train[0].shape)
    
    print(model.summary())

    return model

In [ ]:
def set_nontrainable_layers(model):
  # Set the first layers to be untrainable
  model.trainable = False
        
  return model

In [ ]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = set_nontrainable_layers(model)    
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(30, activation='relu')
    prediction_layer = layers.Dense(1, activation='linear')
    
    model_w_layers = models.Sequential([
        model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
  
    return model_w_layers

In [ ]:
def build_model(model_selected):
  
  model = load_model(model_selected)
  model = set_nontrainable_layers(model)
  model_full = add_last_layers(model)
  model_full.compile(loss="mae", optimizer='adam', metrics='mae')
    
  return model_full

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=20,
    width_shift_range=0,
    height_shift_range=0,
    horizontal_flip=True,
    validation_split=0.2)
datagen.fit(X_train)

In [ ]:
model = Sequential()
model_built = build_model(model_selected)

es = EarlyStopping(monitor = 'val_loss', 
                   patience = 10, 
                   verbose = 1, 
                   restore_best_weights = True)

In [ ]:
history = model_built.fit(datagen.flow(X_train, y_train, batch_size=32) \
                          , validation_data=(X_val, y_val), epochs=50, batch_size=16, callbacks=[es])

In [ ]:
print(model_built.evaluate(X_test, y_test, verbose=1))